In [ ]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
import joblib
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
stopwords_english = set(stopwords.words('english'))
stopwords_tamil = set(['அங்கு', 'அங்கே', 'அடுத்த', 'அதனால்', 'அதன்', 'அதற்கு', 'அதிக', 'அதில்', 'அது', 'அதே', 'அதை', 'அந்த', 'அந்தக்', 'அந்தப்', 'அன்று', 'அல்லது', 'அவன்', 'அவரது', 'அவர்', 'அவர்கள்', 'அவள்', 'அவை', 'ஆகிய', 'ஆகியோர்', 'ஆகும்', 'இங்கு', 'இங்கே', 'இடத்தில்', 'இடம்', 'இதனால்', 'இதனை', 'இதன்', 'இதற்கு', 'இதில்', 'இது', 'இதை', 'இந்த', 'இந்தக்', 'இந்தத்', 'இந்தப்', 'இன்னும்', 'இப்போது', 'இரு', 'இருக்கும்', 'இருந்த', 'இருந்தது', 'இருந்து', 'இவர்', 'இவை', 'உன்', 'உள்ள', 'உள்ளது', 'உள்ளன', 'எந்த', 'என', 'எனக்', 'எனக்கு', 'எனப்படும்', 'எனவும்', 'எனவே', 'எனினும்', 'எனும்', 'என்', 'என்ன', 'என்னும்', 'என்பது', 'என்பதை', 'என்ற', 'என்று', 'என்றும்', 'எல்லாம்', 'ஏன்', 'ஒரு', 'ஒரே', 'ஓர்', 'கொண்ட', 'கொண்டு', 'கொள்ள', 'சற்று', 'சிறு', 'சில', 'சேர்ந்த', 'தனது', 'தன்', 'தவிர', 'தான்', 'நான்', 'நாம்', 'நீ', 'பற்றி', 'பற்றிய', 'பல', 'பலரும்', 'பல்வேறு', 'பின்', 'பின்னர்', 'பிற', 'பிறகு', 'பெரும்', 'பேர்', 'போது', 'போன்ற', 'போல', 'போல்', 'மட்டுமே', 'மட்டும்', 'மற்ற', 'மற்றும்', 'மிக', 'மிகவும்', 'மீது', 'முதல்', 'முறை', 'மேலும்', 'மேல்', 'யார்', 'வந்த', 'வந்து', 'வரும்', 'வரை', 'வரையில்', 'விட', 'விட்டு', 'வேண்டும்', 'வேறு'])
stopwords_combined = stopwords_english.union(stopwords_tamil)

In [ ]:
# Preprocessing function for Tanglish
def preprocess_tanglish(text):
    text = text.lower()
    text = re.sub(r'[^a-zA-Z0-9\s\u0B80-\u0BFF]', '', text)
    text_tokens = text.split()
    text_tokens = [word for word in text_tokens if word not in stopwords_combined]


    return ' '.join(text_tokens)

# Load datasets
df_train = pd.read_csv("/content/Tam-SA-train.csv")
df_valid = pd.read_csv("/content/Tam-SA-val.csv")
df_test = pd.read_csv("/content/Tam-SA-test-without-labels.csv")

# Apply preprocessing
df_train['Text'] = df_train['Text'].apply(preprocess_tanglish)
df_valid['Text'] = df_valid['Text'].apply(preprocess_tanglish)
df_test['Text'] = df_test['Text'].apply(preprocess_tanglish)

In [ ]:
# Step 1: Feature Extraction using TF-IDF Vectorizer
vectorizer = TfidfVectorizer(max_features=5000)
X_train = vectorizer.fit_transform(df_train['Text'])
X_valid = vectorizer.transform(df_valid['Text'])
X_test = vectorizer.transform(df_test['Text'])

# Step 2: Prepare target labels for training and validation
y_train = df_train['Label']
y_valid = df_valid['Label']


In [ ]:
# Step 3: Initialize SVM classifier
svm_classifier = SVC(kernel='linear', C=1, class_weight='balanced')

# Step 4: Train the model using the training data
svm_classifier.fit(X_train, y_train)

# Step 5: Validate the model using the validation data
y_valid_pred = svm_classifier.predict(X_valid)

In [ ]:
# Evaluate the model on validation data
print("Validation Accuracy:", accuracy_score(y_valid, y_valid_pred))
print("Validation Classification Report:")
print(classification_report(y_valid, y_valid_pred))

Validation Accuracy: 0.6325787145459276
Validation Classification Report:
                precision    recall  f1-score   support

Mixed_feelings       0.23      0.04      0.07       472
      Negative       0.52      0.30      0.38       480
      Positive       0.67      0.91      0.77      2272
 unknown_state       0.48      0.33      0.39       619

      accuracy                           0.63      3843
     macro avg       0.48      0.40      0.40      3843
  weighted avg       0.57      0.63      0.58      3843



In [ ]:
# Step 6: Test the model using the test data (No labels in test data)
y_test_pred = svm_classifier.predict(X_test)
df_test['Predicted_Label'] = y_test_pred

# Save the predictions to a CSV file
df_test.to_csv('tamil_test_predictions_svm.csv', index=False)

# Optionally, print a few predictions
print(df_test[['Id', 'Text', 'Predicted_Label']].head())

         Id                                               Text Predicted_Label
0  SA_Ta_01  thalaivan oruvane thalapathy matume mass thala...        Positive
1  SA_Ta_02  kandipaga tamilnadu arasiyal kizhikaraa padam ...        Positive
2  SA_Ta_03  ajith actinghaha vijay actinghahahahaaha hahah...        Positive
3  SA_Ta_04  உண்மையா லவ் பண்ணி வீட்ல சம்மதிக்கலனா பண்ண பொண்...  Mixed_feelings
4  SA_Ta_05  chiiiiiii mokka punda trailer wigpathy sura 2 ...        Negative
